<a href="https://colab.research.google.com/github/giannisgian/Teliki-ergasia/blob/main/Ukraine_Russia_hashtag1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install Twython
import pandas as pd
import numpy as np 
from twython import Twython
from twython import TwythonError
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import ast
import time
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from collections import Counter
from nltk.corpus import stopwords
from nltk.util import ngrams
from sklearn.feature_extraction.text import CountVectorizer
plt.style.use('ggplot')

CONSUMER_KEY = "DyoTjohC4arsxLFBjmh71l5Fi"
CONSUMER_SECRET = "sWE3WV27dOOsZnRpSo19bZCojZykVqftYh1uAQvCwqviro4jT1"
OAUTH_TOKEN = "1133494860554604545-aQrvZDpNyxgi6MwQbHegX8Bkx6CUfI"
OAUTH_TOKEN_SECRET = "inqueIKTVIqqKGXKiHS7dOwNvSmCQdWgYN83ih1kQkNkS"
twitter = Twython(
    CONSUMER_KEY, CONSUMER_SECRET,
    OAUTH_TOKEN, OAUTH_TOKEN_SECRET)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


#  #Ουκρανία

In [ ]:
tweets = []
MAX_ATTEMPTS = 50
COUNT_OF_TWEETS_TO_BE_FETCHED = 10000 

for i in range(0,MAX_ATTEMPTS):

    if(COUNT_OF_TWEETS_TO_BE_FETCHED < len(tweets)):
        print("We Got ", len(tweets), " tweets!!!")
        break # we got the  tweets... !!

    #----------------------------------------------------------------#
    # STEP 1: Query Twitter
    # STEP 2: Save the returned tweets
    # STEP 3: Get the next max_id
    #----------------------------------------------------------------#

    # STEP 1: Query Twitter
    if(0 == i):
        # Query twitter for data. 
        results = twitter.search(q="ουκρανία",count='100')
        print("I Got:", len(results['statuses']), " tweets")
    else:
        # After the first call we should have max_id from result of previous call. Pass it in query.
        results = twitter.search(q="ουκρανία",count='100', include_entities='true',max_id=next_max_id)
        print("I Got:", len(results['statuses']), " tweets")


    # STEP 2: Save the returned tweets
    for result in results['statuses']:
#        tweet_text = result['text']
        tweets.append(result)


    # STEP 3: Get the next max_id
    try:
        # Parse the data returned to get max_id to be passed in consequent call.
        next_results_url_params = results['search_metadata']['next_results']
        next_max_id = next_results_url_params.split('max_id=')[1].split('&')[0]
    except:
        # No more next pages
        break

In [ ]:
#results['search_metadata']

In [ ]:
#results['statuses']

In [ ]:
Uk_df = pd.DataFrame(tweets)
Uk_df

In [ ]:
# Number of tweets the user has made
print("I got :", len(Uk_df), " in total!!!")

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
Uk_df.to_csv("/content/gdrive/My Drive/Colab Notebooks/Ουκρανία_tweet.tsv", sep='\t')

# Νέα ενότητα #Ουκρανία

In [ ]:
Uk = Uk_df[['created_at','text' ]]
Uk = Uk.rename(columns={'created_at': 'date'} )
Uk

In [ ]:
Uk.dtypes

In [ ]:
Uk['year'] = pd.DatetimeIndex(Uk['date']).year
Uk['month'] = pd.DatetimeIndex(Uk['date']).month
Uk['day'] = pd.DatetimeIndex(Uk['date']).day
Uk

In [ ]:
Uk['text'] = Uk['text'].str.replace(r'https?:\/\/.*[\r\n]*'," ")

In [ ]:
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt

In [ ]:
Uk[Uk['year']==2022]['text']

In [ ]:
# Ισως καποτε χρειαστεί να αφαιρέσουμε τους τόνους.....
text = Uk['text'].str.replace('ά','α').str.replace("έ", "ε").str.replace('ή','η').str.replace('ί','ι').str.replace('ό','ο').str.replace('ύ','υ').str.replace('ώ','ω')

In [ ]:
text = Uk[Uk['year']==2022]['text'].str.cat(sep = '.').replace("amp", ' ')
text

In [ ]:
list(STOPWORDS)[0:20]

In [ ]:
import spacy
!python -m spacy download el_core_news_sm

In [ ]:
nlp = spacy.load('el_core_news_sm')
list(nlp.Defaults.stop_words)[0:20]

In [ ]:
text = Uk['text'].str.cat(sep = '.').replace('.','').strip()
text = text.replace('amp','').strip()
text = text.replace('rt','').strip()
text = text.replace('\n','').strip()
text = text.replace(':','').strip()
text

In [ ]:
from PIL import Image
import numpy as np

In [ ]:
doc = nlp(Uk['text'][1])

In [ ]:
for token in doc:
    print(token, token.lemma_)

In [ ]:
' '.join(token.lemma_ for token in doc)
text

In [ ]:
full_doc = nlp(text)

In [ ]:
clear_text = ' '.join(token.lemma_ for token in full_doc)
clear_text[200:400]

In [ ]:
wordcloud = WordCloud(
    stopwords = nlp.Defaults.stop_words,
    width = 2000,
    height = 1000,
    background_color = 'yellow'
 ).generate(clear_text)
fig = plt.figure(
    figsize = (40, 30),
    facecolor = 'k',
    edgecolor = 'k')
plt.imshow(wordcloud, interpolation = 'bilinear')
plt.axis('off')
plt.tight_layout(pad=0)
plt.show()

In [ ]:
from collections import Counter
Counter(" ".join(Uk["text"]).split()).most_common(16)

In [ ]:
for word in nlp.Defaults.stop_words:
  print(word)
  break

In [ ]:
Uk['text_no_stopwords'] = Uk['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (nlp.Defaults.stop_words) and word != ':' and word != '\n' and word != "amp" and word != 'rt' and word != ',']))
Uk['text_no_stopwords'] = Uk['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (nlp.Defaults.stop_words)]))
Uk['clean_text_no_stopwords'] = Uk['text']
#for stopword in nlp.Defaults.stop_words:
Uk['clean_text_no_stopwords'] = Uk['clean_text_no_stopwords'].apply(lambda x: ' '.join([word for word in x.split() if word not in nlp.Defaults.stop_words and word != "ή" and word != "ία" and word != "κι"]))
Uk['clean_text_no_stopwords'] = Uk['clean_text_no_stopwords'].apply(lambda x: x.replace("amp", "").strip())
Uk['clean_text_no_stopwords'] = Uk['clean_text_no_stopwords'].apply(lambda x: x.replace("rt", "").strip())
Uk['clean_text_no_stopwords'] = Uk['clean_text_no_stopwords'].apply(lambda x: x.replace(":", "").strip())
Uk['clean_text_no_stopwords'] = Uk['clean_text_no_stopwords'].apply(lambda x: x.replace(",", "").strip())
Uk['clean_text_no_stopwords'] = Uk['clean_text_no_stopwords'].apply(lambda x: x.replace("\n", "").strip())
Uk['clean_text_no_stopwords'] = Uk['clean_text_no_stopwords'].apply(lambda x: x.replace(".", "").strip())

In [ ]:
freq = Counter(" ".join(Uk["clean_text_no_stopwords"]).split()).most_common(16)
freq

In [ ]:
names, values = zip(*freq)
# names = [x[0] for x in data]  # These two lines are equivalent to the the zip-command.
# values = [x[1] for x in data] # These two lines are equivalent to the the zip-command.

ind = np.arange(len(freq))  # the x locations for the groups
width = 0.35       # the width of the bars

fig, ax = plt.subplots(figsize=(20,5))
rects1 = ax.bar(ind, values, width, color='r')
# add some text for labels, title and axes ticks
ax.set_ylabel('Count')
ax.set_xticks(ind+width/10.)
ax.set_xticklabels(names)

def autolabel(rects):
    # attach some text labels
    for rect in rects:
        height = rect.get_height()
        ax.text(rect.get_x() + rect.get_width()/2, height,
                '%d' % int(height),
                ha='center', va='bottom')

autolabel(rects1)

plt.show()

# Emolex Ουκρανία

In [ ]:
filepath = "https://raw.githubusercontent.com/datajour-gr/Data_journalism/master/week10/NRC_GREEK_Translated_6_2020.csv"
emolex_df = pd.read_csv(filepath)
emolex_df.head()

In [ ]:
emolex_df.tail()

In [ ]:
emolex_df = emolex_df.drop_duplicates(subset=['word'])
emolex_df = emolex_df.dropna()
emolex_df.reset_index(drop = True, inplace = True)

In [ ]:
emolex_df.tail()

In [ ]:
import spacy
nlp = spacy.load('el_core_news_sm')

In [ ]:
len(Uk)
Uk

In [ ]:
#short_tweets_Ukr = Uk[['created_at','full_text' ]]
#short_tweets_Ukr = short_tweets_Ukr.rename(columns={'created_at': 'date', 'full_text': 'text'} )

In [ ]:
short_tweets_Ukr = Uk[['date','text' ]]

short_tweets_Ukr

In [ ]:
short_tweets_Ukr.tail()

In [ ]:
short_tweets_Ukr.dtypes

In [ ]:
short_tweets_Ukr['date'] = pd.to_datetime(short_tweets_Ukr['date'], format='%a %b %d %H:%M:%S +0000 %Y')

In [ ]:
short_tweets_Ukr['year'] = pd.DatetimeIndex(short_tweets_Ukr['date']).year
short_tweets_Ukr['month'] = pd.DatetimeIndex(short_tweets_Ukr['date']).month
short_tweets_Ukr['day'] = pd.DatetimeIndex(short_tweets_Ukr['date']).day
short_tweets_Ukr

In [ ]:
short_tweets_Ukr.dtypes

In [ ]:
emolex_df['word'].head(3)

In [ ]:
short_tweets_Ukr['text'] = short_tweets_Ukr['text'].str.replace(r'https?:\/\/.*[\r\n]*'," ")

In [ ]:
short_tweets_Ukr.head()

In [ ]:
vec = CountVectorizer(analyzer = 'word', vocabulary = emolex_df.word, 
                      strip_accents = 'unicode',  
                      stop_words= nlp.Defaults.stop_words,
                      ngram_range=(1, 2))

In [ ]:
#from sklearn.feature_extraction.text import TfidfVectorizer
#CountVectorizer(stop_words='english', binary=True)
# Θέλω να ελέγξω μόνο τις λέξεις που υπάρχουν στο συναισθηματικό λεξικό

#CountVectorizer(analyzer = 'word', 
#              strip_accents = 'unicode', 
#               token_pattern='[Α-Ωα-ωΆ-Ώά-ώ]{2,}', 
#               stop_words= lc_stop_words_df['stop_w'].tolist())


matrix = vec.fit_transform(short_tweets_Ukr['text'])
vocab = vec.get_feature_names()
wordcount_df = pd.DataFrame(matrix.toarray(), columns=vocab)
wordcount_df.head()

In [ ]:
wordcount_df.shape

In [ ]:
emolex_df.head()

In [ ]:
emolex_df[emolex_df.Anger == 1].tail()

In [ ]:
# Φτιάξε μια λίστα με positive words

angry_words = emolex_df[emolex_df.Anger == 1]['word']

positive_words = emolex_df[emolex_df.Positive == 1]['word']


# Φτιάξε μια λίστα με sadness words
sadness_words = emolex_df[emolex_df.Sadness == 1]['word']


# Φτιάξε μια λίστα με surprise words
surprise_words = emolex_df[emolex_df.Surprise == 1]['word']


# Φτιάξε μια λίστα με disgust words
disgust_words = emolex_df[emolex_df.Disgust == 1]['word']


# Φτιάξε μια λίστα με anticipation words
anticipation_words = emolex_df[emolex_df.Anticipation == 1]['word']


# Φτιάξε μια λίστα με negative words
negative_words = emolex_df[emolex_df.Negative == 1]['word']



# Φτιάξε μια λίστα με joy words
joy_words = emolex_df[emolex_df.Joy == 1]['word']


# Φτιάξε μια λίστα με trust words
trust_words = emolex_df[emolex_df.Trust == 1]['word']



# Φτιάξε μια λίστα με fear words
fear_words = emolex_df[emolex_df.Fear == 1]['word']


In [ ]:
short_tweets_Ukr.head(3)

In [ ]:
short_tweets_Ukr['anger'] = wordcount_df[angry_words].sum(axis=1)

short_tweets_Ukr['positivity'] = wordcount_df[positive_words].sum(axis=1)


short_tweets_Ukr['joy'] = wordcount_df[joy_words].sum(axis=1)


short_tweets_Ukr['disgust'] = wordcount_df[disgust_words].sum(axis=1)


short_tweets_Ukr['surprise'] = wordcount_df[surprise_words].sum(axis=1)

short_tweets_Ukr['trust'] = wordcount_df[trust_words].sum(axis=1)


short_tweets_Ukr['anticipation'] = wordcount_df[anticipation_words].sum(axis=1)


short_tweets_Ukr['sadness'] = wordcount_df[sadness_words].sum(axis=1)

short_tweets_Ukr['negative'] = wordcount_df[negative_words].sum(axis=1)

short_tweets_Ukr['fear'] = wordcount_df[fear_words].sum(axis=1)


In [ ]:
short_tweets_Ukr

In [ ]:
max = 0
happiest_tweet = ""
for i in range(0,len(short_tweets_Ukr)):
  #print(short_tweets_Rus.iloc[i]['positivity'])
  if short_tweets_Ukr.iloc[i]['positivity'] > max:
    happiest_tweet = short_tweets_Ukr.iloc[i]['text']
    max = short_tweets_Ukr.iloc[i]['positivity']


happiest_tweet

In [ ]:
max = 0
negative_tweet = ""
for i in range(0,len(short_tweets_Ukr)):
  #print(short_tweets_Ukr.iloc[i]['positivity'])
  if short_tweets_Ukr.iloc[i]['negative'] > max:
    negative_tweet = short_tweets_Ukr.iloc[i]['text']
    max = short_tweets_Ukr.iloc[i]['negative']


negative_tweet

In [ ]:
max = 0
anger_tweet = ""
for i in range(0,len(short_tweets_Ukr)):
  #print(short_tweets_Ukr.iloc[i]['positivity'])
  if short_tweets_Ukr.iloc[i]['anger'] > max:
    anger_tweet = short_tweets_Ukr.iloc[i]['text']
    max = short_tweets_Ukr.iloc[i]['anger']

anger_tweet

In [ ]:
max = 0
saddest_tweet = ""
for i in range(0,len(short_tweets_Ukr)):
  #print(short_tweets_Ukr.iloc[i]['positivity'])
  if short_tweets_Ukr.iloc[i]['sadness'] > max:
    saddest_tweet = short_tweets_Ukr.iloc[i]['text']
    max = short_tweets_Ukr.iloc[i]['sadness']

saddest_tweet

In [ ]:
max = 0
fear_tweet = ""
for i in range(0,len(short_tweets_Ukr)):
  #print(short_tweets_Ukr.iloc[i]['positivity'])
  if short_tweets_Ukr.iloc[i]['fear'] > max:
    fear_tweet = short_tweets_Ukr.iloc[i]['text']
    max = short_tweets_Ukr.iloc[i]['fear']

fear_tweet

In [ ]:
max = 0
trust_tweet = ""
for i in range(0,len(short_tweets_Ukr)):
  #print(short_tweets_Ukr.iloc[i]['positivity'])
  if short_tweets_Ukr.iloc[i]['trust'] > max:
    trust_tweet = short_tweets_Ukr.iloc[i]['text']
    max = short_tweets_Ukr.iloc[i]['trust']

trust_tweet

In [ ]:
short_tweets_Ukr.set_index('date' , inplace=True)

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')

In [ ]:
short_tweets_Ukr['text'].str.len().resample('D').mean().plot() 

In [ ]:
short_tweets_Ukr['positivity'].resample('D').mean().plot() 

In [ ]:
short_tweets_Ukr['negative'].resample('D').sum().plot(color = 'b',label = 'neg').legend() 

In [ ]:
short_tweets_Ukr['anger'].resample('D').sum().plot(figsize=(16,4))

In [ ]:
ax = short_tweets_Ukr['positivity'].resample('D').mean().plot(figsize=(16,4), 
                                                          color = 'g', label='pos')
short_tweets_Ukr['negative'].resample('D').mean().plot(figsize=(16,4), ax = ax, 
                                                   label = 'neg', color = 'r')
short_tweets_Ukr['trust'].resample('D').mean().plot(figsize=(16,4), ax = ax, 
                                                label = 'trust',color = 'blue').legend()

In [ ]:
plt.style.use('ggplot')
ax = short_tweets_Ukr['positivity'].resample('D').mean().plot(figsize=(16,4), 
                                                          color = 'g', label='pos')
short_tweets_Ukr['negative'].resample('D').mean().plot(figsize=(16,4), ax = ax, 
                                                   label = 'neg', color = 'r')
short_tweets_Ukr['trust'].resample('D').mean().plot(figsize=(16,4), ax = ax, 
                                                label = 'trust',color = 'blue').legend()

short_tweets_Ukr['disgust'].resample('D').mean().plot(figsize=(16,4), ax = ax, 
                                                label = 'disg',color = 'black').legend() 

short_tweets_Ukr['anger'].resample('D').mean().plot(figsize=(16,4), ax = ax, 
                                                label = 'anger',color = 'purple').legend() 

short_tweets_Ukr['fear'].resample('D').mean().plot(figsize=(16,4), ax = ax, 
                                                label = 'fear',color = 'brown').legend()    

In [ ]:
short_tweets_Ukr['text'].groupby(by=short_tweets_Ukr.index.hour).count().plot(kind='barh')
plt.title("Posts per hour")
plt.xlabel("Number of posts")
plt.ylabel("Hour of the day")

#  #Ρωσία

In [ ]:
tweets = []
MAX_ATTEMPTS = 50
COUNT_OF_TWEETS_TO_BE_FETCHED = 10000 

for i in range(0,MAX_ATTEMPTS):

    if(COUNT_OF_TWEETS_TO_BE_FETCHED < len(tweets)):
        print("We Got ", len(tweets), " tweets!!!")
        break # we got the  tweets... !!

    #----------------------------------------------------------------#
    # STEP 1: Query Twitter
    # STEP 2: Save the returned tweets
    # STEP 3: Get the next max_id
    #----------------------------------------------------------------#

    # STEP 1: Query Twitter
    if(0 == i):
        # Query twitter for data. 
        results = twitter.search(q="Ρωσία",count='100')
        print("I Got:", len(results['statuses']), " tweets")
    else:
        # After the first call we should have max_id from result of previous call. Pass it in query.
        results = twitter.search(q="ρωσία",count='100', include_entities='true',max_id=next_max_id)
        print("I Got:", len(results['statuses']), " tweets")


    # STEP 2: Save the returned tweets
    for result in results['statuses']:
#        tweet_text = result['text']
        tweets.append(result)


    # STEP 3: Get the next max_id
    try:
        # Parse the data returned to get max_id to be passed in consequent call.
        next_results_url_params = results['search_metadata']['next_results']
        next_max_id = next_results_url_params.split('max_id=')[1].split('&')[0]
    except:
        # No more next pages
        break

In [ ]:
#results['search_metadata']

In [ ]:
#results['statuses']

In [ ]:
Ru_df = pd.DataFrame(tweets)
Ru_df

In [ ]:
# Number of tweets the user has made
print("I got :", len(Ru_df), " in total!!!")

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
Ru_df.to_csv("/content/gdrive/My Drive/Colab Notebooks/Ρωσία_tweet.tsv", sep='\t')

# Νέα ενότητα #Ρωσία

In [ ]:
Ru_df =pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/Ρωσία_tweet.tsv", sep='\t')

In [ ]:
Ru = Ru_df[['created_at','text' ]]
Ru = Ru.rename(columns={'created_at': 'date'} )
Ru

In [ ]:
#Ru.dtypes

In [ ]:
Ru['year'] = pd.DatetimeIndex(Ru['date']).year
Ru['month'] = pd.DatetimeIndex(Ru['date']).month
Ru['day'] = pd.DatetimeIndex(Ru['date']).day
Ru

In [ ]:
Ru['text'] = Ru['text'].str.replace(r'https?:\/\/.*[\r\n]*'," ")

In [ ]:
Ru['day'].value_counts().plot(kind='bar')

In [ ]:
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt

In [ ]:
Ru[Ru['year']==2022]['text']

In [ ]:
# Ισως καποτε χρειαστεί να αφαιρέσουμε τους τόνους.....
text = Ru['text'].str.replace('ά','α').str.replace("έ", "ε").str.replace('ή','η').str.replace('ί','ι').str.replace('ό','ο').str.replace('ύ','υ').str.replace('ώ','ω')

In [ ]:
text = Ru[Ru['year']==2022]['text'].str.cat(sep = '.').replace("amp", ' ')
text

In [ ]:
list(STOPWORDS)[0:20]

In [ ]:
import spacy
!python -m spacy download el_core_news_sm

In [ ]:
nlp = spacy.load('el_core_news_sm')
list(nlp.Defaults.stop_words)[0:20]

In [ ]:
text = Ru['text'].str.cat(sep = '.').replace('.','').strip()
text = text.replace('amp','').strip()
text = text.replace('rt','').strip()
text = text.replace('\n','').strip()
text = text.replace(':','').strip()
text

In [ ]:
from PIL import Image
import numpy as np

In [ ]:
doc = nlp(Ru['text'][1])

In [ ]:
for token in doc:
    print(token, token.lemma_)

In [ ]:
' '.join(token.lemma_ for token in doc)
text

In [ ]:
full_doc = nlp(text)

In [ ]:
clear_text = ' '.join(token.lemma_ for token in full_doc)
clear_text[200:400]

In [ ]:
wordcloud = WordCloud(
    stopwords = nlp.Defaults.stop_words,
    width = 2000,
    height = 1000,
    background_color = 'red'
 ).generate(clear_text)
fig = plt.figure(
    figsize = (40, 30),
    facecolor = 'k',
    edgecolor = 'k')
plt.imshow(wordcloud, interpolation = 'bilinear')
plt.axis('off')
plt.tight_layout(pad=0)
plt.show()

In [ ]:
from collections import Counter
Counter(" ".join(Ru["text"]).split()).most_common(16)

In [ ]:
for word in nlp.Defaults.stop_words:
  print(word)
  break

In [ ]:
Ru['text_no_stopwords'] = Ru['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (nlp.Defaults.stop_words) and word != ':' and word != '\n' and word != "amp" and word != 'rt' and word != ',']))
Ru['text_no_stopwords'] = Ru['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (nlp.Defaults.stop_words)]))
Ru['clean_text_no_stopwords'] = Ru['text']
#for stopword in nlp.Defaults.stop_words:
Ru['clean_text_no_stopwords'] = Ru['clean_text_no_stopwords'].apply(lambda x: ' '.join([word for word in x.split() if word not in nlp.Defaults.stop_words and word != "ή" and word != "ία" and word != "κι"]))
Ru['clean_text_no_stopwords'] = Ru['clean_text_no_stopwords'].apply(lambda x: x.replace("amp", "").strip())
Ru['clean_text_no_stopwords'] = Ru['clean_text_no_stopwords'].apply(lambda x: x.replace("rt", "").strip())
Ru['clean_text_no_stopwords'] = Ru['clean_text_no_stopwords'].apply(lambda x: x.replace(":", "").strip())
Ru['clean_text_no_stopwords'] = Ru['clean_text_no_stopwords'].apply(lambda x: x.replace(",", "").strip())
Ru['clean_text_no_stopwords'] = Ru['clean_text_no_stopwords'].apply(lambda x: x.replace("\n", "").strip())
Ru['clean_text_no_stopwords'] = Ru['clean_text_no_stopwords'].apply(lambda x: x.replace(".", "").strip())

In [ ]:
freq = Counter(" ".join(Ru["clean_text_no_stopwords"]).split()).most_common(16)
freq

In [ ]:
names, values = zip(*freq)
# names = [x[0] for x in data]  # These two lines are equivalent to the the zip-command.
# values = [x[1] for x in data] # These two lines are equivalent to the the zip-command.

ind = np.arange(len(freq))  # the x locations for the groups
width = 0.35       # the width of the bars

fig, ax = plt.subplots(figsize=(20,5))
rects1 = ax.bar(ind, values, width, color='r')
# add some text for labels, title and axes ticks
ax.set_ylabel('Count')
ax.set_xticks(ind+width/10.)
ax.set_xticklabels(names)

def autolabel(rects):
    # attach some text labels
    for rect in rects:
        height = rect.get_height()
        ax.text(rect.get_x() + rect.get_width()/2, height,
                '%d' % int(height),
                ha='center', va='bottom')

autolabel(rects1)

plt.show()

# Emolex Ρωσία

In [ ]:
filepath = "https://raw.githubusercontent.com/datajour-gr/Data_journalism/master/week10/NRC_GREEK_Translated_6_2020.csv"
emolex_df = pd.read_csv(filepath)
emolex_df.head()

In [ ]:
emolex_df.tail()

In [ ]:
emolex_df = emolex_df.drop_duplicates(subset=['word'])
emolex_df = emolex_df.dropna()
emolex_df.reset_index(drop = True, inplace = True)

In [ ]:
emolex_df.tail()

In [ ]:
import spacy
nlp = spacy.load('el_core_news_sm')

In [ ]:
len(Ru)
Ru

In [ ]:
short_tweets_Ru = Ru[['date','text' ]]

short_tweets_Ru

In [ ]:
short_tweets_Ru.tail()

In [ ]:
short_tweets_Ru.dtypes

In [ ]:
short_tweets_Ru['date'] = pd.to_datetime(short_tweets_Ru['date'], format='%a %b %d %H:%M:%S +0000 %Y')

In [ ]:
short_tweets_Ru['year'] = pd.DatetimeIndex(short_tweets_Ru['date']).year
short_tweets_Ru['month'] = pd.DatetimeIndex(short_tweets_Ru['date']).month
short_tweets_Ru['day'] = pd.DatetimeIndex(short_tweets_Ru['date']).day
short_tweets_Ru

In [ ]:
short_tweets_Ru.dtypes

In [ ]:
emolex_df['word'].head(3)

In [ ]:
short_tweets_Ru['text'] = short_tweets_Ru['text'].str.replace(r'https?:\/\/.*[\r\n]*'," ")

In [ ]:
short_tweets_Ru.head()

In [ ]:
vec = CountVectorizer(analyzer = 'word', vocabulary = emolex_df.word, 
                      strip_accents = 'unicode',  
                      stop_words= nlp.Defaults.stop_words,
                      ngram_range=(1, 2))

In [ ]:
#from sklearn.feature_extraction.text import TfidfVectorizer
#CountVectorizer(stop_words='english', binary=True)
# Θέλω να ελέγξω μόνο τις λέξεις που υπάρχουν στο συναισθηματικό λεξικό

#CountVectorizer(analyzer = 'word', 
#              strip_accents = 'unicode', 
#               token_pattern='[Α-Ωα-ωΆ-Ώά-ώ]{2,}', 
#               stop_words= lc_stop_words_df['stop_w'].tolist())


matrix = vec.fit_transform(short_tweets_Ru['text'])
vocab = vec.get_feature_names()
wordcount_df = pd.DataFrame(matrix.toarray(), columns=vocab)
wordcount_df.head()

In [ ]:
wordcount_df.shape

In [ ]:
emolex_df.head()

In [ ]:
emolex_df[emolex_df.Anger == 1].tail()

In [ ]:
# Φτιάξε μια λίστα με positive words

angry_words = emolex_df[emolex_df.Anger == 1]['word']

positive_words = emolex_df[emolex_df.Positive == 1]['word']


# Φτιάξε μια λίστα με sadness words
sadness_words = emolex_df[emolex_df.Sadness == 1]['word']


# Φτιάξε μια λίστα με surprise words
surprise_words = emolex_df[emolex_df.Surprise == 1]['word']


# Φτιάξε μια λίστα με disgust words
disgust_words = emolex_df[emolex_df.Disgust == 1]['word']


# Φτιάξε μια λίστα με anticipation words
anticipation_words = emolex_df[emolex_df.Anticipation == 1]['word']


# Φτιάξε μια λίστα με negative words
negative_words = emolex_df[emolex_df.Negative == 1]['word']



# Φτιάξε μια λίστα με joy words
joy_words = emolex_df[emolex_df.Joy == 1]['word']


# Φτιάξε μια λίστα με trust words
trust_words = emolex_df[emolex_df.Trust == 1]['word']



# Φτιάξε μια λίστα με fear words
fear_words = emolex_df[emolex_df.Fear == 1]['word']


In [ ]:
short_tweets_Ru.head(3)

In [ ]:
short_tweets_Ru['anger'] = wordcount_df[angry_words].sum(axis=1)

short_tweets_Ru['positivity'] = wordcount_df[positive_words].sum(axis=1)


short_tweets_Ru['joy'] = wordcount_df[joy_words].sum(axis=1)


short_tweets_Ru['disgust'] = wordcount_df[disgust_words].sum(axis=1)


short_tweets_Ru['surprise'] = wordcount_df[surprise_words].sum(axis=1)

short_tweets_Ru['trust'] = wordcount_df[trust_words].sum(axis=1)


short_tweets_Ru['anticipation'] = wordcount_df[anticipation_words].sum(axis=1)


short_tweets_Ru['sadness'] = wordcount_df[sadness_words].sum(axis=1)

short_tweets_Ru['negative'] = wordcount_df[negative_words].sum(axis=1)

short_tweets_Ru['fear'] = wordcount_df[fear_words].sum(axis=1)


In [ ]:
short_tweets_Ru

In [ ]:
max = 0
happiest_tweet = ""
for i in range(0,len(short_tweets_Ru)):
  #print(short_tweets_Ru.iloc[i]['positivity'])
  if short_tweets_Ru.iloc[i]['positivity'] > max:
    happiest_tweet = short_tweets_Ru.iloc[i]['text']
    max = short_tweets_Ru.iloc[i]['positivity']


happiest_tweet

In [ ]:
max = 0
negative_tweet = ""
for i in range(0,len(short_tweets_Ru)):
  #print(short_tweets_Ru.iloc[i]['positivity'])
  if short_tweets_Ru.iloc[i]['negative'] > max:
    negative_tweet = short_tweets_Ru.iloc[i]['text']
    max = short_tweets_Ru.iloc[i]['negative']


negative_tweet

In [ ]:
max = 0
saddest_tweet = ""
for i in range(0,len(short_tweets_Ru)):
  #print(short_tweets_Ru.iloc[i]['positivity'])
  if short_tweets_Ru.iloc[i]['sadness'] > max:
    saddest_tweet = short_tweets_Ru.iloc[i]['text']
    max = short_tweets_Ru.iloc[i]['sadness']

saddest_tweet

In [ ]:
max = 0
fear_tweet = ""
for i in range(0,len(short_tweets_Ru)):
  #print(short_tweets_Ru.iloc[i]['positivity'])
  if short_tweets_Ru.iloc[i]['fear'] > max:
    fear_tweet = short_tweets_Ru.iloc[i]['text']
    max = short_tweets_Ru.iloc[i]['fear']

fear_tweet

In [ ]:
max = 0
trust_tweet = ""
for i in range(0,len(short_tweets_Ru)):
  #print(short_tweets_Ru.iloc[i]['positivity'])
  if short_tweets_Ru.iloc[i]['trust'] > max:
    trust_tweet = short_tweets_Ru.iloc[i]['text']
    max = short_tweets_Ru.iloc[i]['trust']

trust_tweet

In [ ]:
short_tweets_Ru.set_index('date' , inplace=True)

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')

In [ ]:
short_tweets_Ru['text'].str.len().groupby(by=short_tweets_Ru.index.month).mean().plot()

In [ ]:
short_tweets_Ru['text'].str.len().resample('D').mean().plot() 

In [ ]:
short_tweets_Ru['positivity'].resample('D').mean().plot() 

In [ ]:
short_tweets_Ru['negative'].resample('D').sum().plot(color = 'b',label = 'neg').legend() 

In [ ]:
short_tweets_Ru['anger'].resample('D').sum().plot(figsize=(16,4))

In [ ]:
ax = short_tweets_Ru['positivity'].resample('D').mean().plot(figsize=(16,4), 
                                                          color = 'g', label='pos')
short_tweets_Ru['negative'].resample('D').mean().plot(figsize=(16,4), ax = ax, 
                                                   label = 'neg', color = 'r')
short_tweets_Ru['trust'].resample('D').mean().plot(figsize=(16,4), ax = ax, 
                                                label = 'trust',color = 'blue').legend()

In [ ]:
plt.style.use('ggplot')
ax = short_tweets_Ru['positivity'].resample('D').mean().plot(figsize=(16,4), 
                                                          color = 'g', label='pos')
short_tweets_Ru['negative'].resample('D').mean().plot(figsize=(16,4), ax = ax, 
                                                   label = 'neg', color = 'r')
short_tweets_Ru['trust'].resample('D').mean().plot(figsize=(16,4), ax = ax, 
                                                label = 'trust',color = 'blue').legend()

short_tweets_Ru['disgust'].resample('D').mean().plot(figsize=(16,4), ax = ax, 
                                                label = 'disg',color = 'black').legend() 

short_tweets_Ru['anger'].resample('D').mean().plot(figsize=(16,4), ax = ax, 
                                                label = 'anger',color = 'purple').legend() 

short_tweets_Ru['fear'].resample('D').mean().plot(figsize=(16,4), ax = ax, 
                                                label = 'fear',color = 'brown').legend()    

In [ ]:
short_tweets_Ru['text'].groupby(by=short_tweets_Ru.index.hour).count().plot(kind='barh')
plt.title("Posts per hour")
plt.xlabel("Number of posts")
plt.ylabel("Hour of the day")

In [ ]:
short_tweets_Ru

# Rudf - Ukdf

In [ ]:
ax = short_tweets_Ru['positivity'].resample('B').sum().plot(figsize=(16,4), color = 'red')
short_tweets_Ru['negative'].resample('B').sum().plot(figsize=(16,4), ax = ax, color = 'green')
short_tweets_Ru['trust'].resample('B').sum().plot(figsize=(16,4), ax = ax, color = 'blue')
ax = short_tweets_Ukr['positivity'].resample('B').sum().plot(figsize=(16,4), color = 'yellow')
short_tweets_Ukr['negative'].resample('B').sum().plot(figsize=(16,4), ax = ax, color = 'purple')
short_tweets_Ukr['trust'].resample('B').sum().plot(figsize=(16,4), ax = ax, color = 'orange')

In [ ]:
ax = short_tweets_Ru['positivity'].resample('B').mean().plot(figsize=(16,4), 
                                                          color = 'g', label='pos Ru')
ax = short_tweets_Ukr['positivity'].resample('B').mean().plot(figsize=(16,4), 
                                                          color = 'y', label='pos Uk')
short_tweets_Ru['negative'].resample('B').mean().plot(figsize=(16,4), ax = ax, 
                                                   label = 'neg Ru', color = 'r')
short_tweets_Ukr['negative'].resample('B').mean().plot(figsize=(16,4), ax = ax, 
                                                   label = 'neg Uk', color = 'purple')
short_tweets_Ru['trust'].resample('B').mean().plot(figsize=(16,4), ax = ax, 
                                                label = 'trust Ru',color = 'blue').legend()
short_tweets_Ukr['trust'].resample('B').mean().plot(figsize=(16,4), ax = ax, 
                                                label = 'trust Uk',color = 'orange').legend()

In [ ]:
plt.style.use('ggplot')
ax = short_tweets_Ru['fear'].resample('B').mean().plot(figsize=(16,4), 
                                                          color = 'g', label='pos Ru')
short_tweets_Ru['anger'].resample('B').mean().plot(figsize=(16,4), ax = ax, 
                                                   label = 'neg Ru', color = 'r')
short_tweets_Ru['disgust'].resample('B').mean().plot(figsize=(16,4), ax = ax, 
                                                label = 'trust Ru',color = 'blue').legend()    
short_tweets_Ukr['fear'].resample('B').mean().plot(figsize=(16,4), 
                                                          color = 'yellow', label='pos Uk')
short_tweets_Ukr['anger'].resample('B').mean().plot(figsize=(16,4), ax = ax, 
                                                   label = 'neg Uk', color = 'orange')
short_tweets_Ukr['disgust'].resample('B').mean().plot(figsize=(16,4), ax = ax, 
                                                label = 'trust Uk',color = 'purple').legend()                                                